In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from sklearn.neighbors import NearestNeighbors
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.ensemble import BaggingClassifier

In [4]:
#Here I want to test some regressors

import numpy as np
import h5py
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

In [5]:
#Now it turns out my sample is very biased
#about 1% of the dm particles have associated stellar mass
#
#Apparently you can pass a bias to the network and it will
#work with that
#
#First load the data

f_halo = h5py.File('../m12i_res_7100_cdm/halo_600.hdf5')

pos_halo = f_halo['position'][:]
mass_halo = f_halo['mass'][:]
radius_halo = f_halo['radius'][:]
vel_halo = f_halo['velocity'][:]

#identify the host                                                                                                   \
                                                                                                                      
host_id = np.argmax(mass_halo)
host_mass = mass_halo[host_id]
host_pos = pos_halo[host_id]
host_vel = vel_halo[host_id]

f = h5py.File('DM_data_w_stars_training.hdf5')

coords = f['PartType1']['Coordinates'][:]
vel = f['PartType1']['Velocities'][:]
mass_ratio = f['PartType1']['Mass_Ratio'][:]

In [6]:
diff_coord = coords-host_pos
diff_vel = vel - host_vel

dist  = np.linalg.norm(diff_coord,axis=1)
gal_select = (dist<100.0)

diff_coord_gal = diff_coord[gal_select]
diff_vel_gal = diff_vel[gal_select]
mass_ratio_gal = mass_ratio[gal_select]

phase_space_coords =  np.concatenate((diff_coord_gal,diff_vel_gal),axis=1)

print len(dist)
print len(mass_ratio_gal)

print (np.sum(mass_ratio_gal>0.0))

70514272
15489004
342360


In [30]:
X_train, X_test, y_train, y_test = train_test_split(phase_space_coords,mass_ratio_gal,
                                                    test_size=0.99,random_state=102)

In [31]:
y_train_classifier  = np.array((y_train!=0),dtype=int)

neg, pos = np.bincount(y_train_classifier)

print neg, pos

initial_bias = np.log([float(pos)/float(neg)])

print initial_bias

151485 3405
[-3.79524167]


In [32]:
def build_model(output_bias=None):
    output_bias = tf.keras.initializers.Constant(output_bias)
    model = keras.Sequential([layers.Flatten(input_shape=(6,)),
                              layers.Dense(64, activation='relu'),
                              layers.Dense(64, activation='relu'),
                              layers.Dense(1, activation='sigmoid',bias_initializer=output_bias)
                             ])
    
    optimizer = 'adam'

    #Can use BinaryCrossentropy because I have only two labels                                                        
    loss  = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    
    model.compile(loss=loss,optimizer=optimizer,metrics=['accuracy'])
    
    return model

print 'I guess we build this thing'

model = build_model(output_bias=initial_bias)

I guess we build this thing


In [ ]:
EPOCHS = 1000
history = model.fit(X_train, y_train_classifier, epochs=EPOCHS, verbose=0)

In [ ]:
model_output = model.predict(X_test)

print np.sum(model_output>0.0)

#plt.hist(model_output,bins=25,histtype='step')
#plt.show()

1046691


In [ ]:
float(len(model_output))/float(np.sum(model_output>0.0))

14.650086797345157